機械学習特論 第5回課題 k近傍法 SVM で アヤメのデータ、MNIST, Fashion-MNIST の分類を比較

In [1]:
# google colab で実行する場合は、次の行の先頭の # を削除してこのブロックを実行する
#!pip install japanize-matplotlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

In [8]:
# sklearnデータセットに収録されたiris(アヤメ)のデータセットをロードしてデータフレームを作成
def load_iris_data():
    data = load_iris()
    x = pd.DataFrame(data["data"],columns=data["feature_names"])[50:150]
    y = pd.DataFrame(data["target"],columns=["target"])[50:150] # species 1 2 を抽出
    return x, y

# 手書き文字のデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_mnist_data():
    data = fetch_openml('mnist_784', version=1)
    _x = np.array(data['data'].astype(np.float32))
    _y = np.array(data['target'].astype(np.int32))
    _, x, _, y = train_test_split(_x, _y, test_size=0.1, random_state=1, stratify=_y)
    return x, y

# Fashion-MNISTデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_fashion_mnist_data():
    data = fetch_openml('Fashion-MNIST')
    _x = np.array(data['data'].astype(np.float32))
    _y = np.array(data['target'].astype(np.int32))
    _, x, _, y = train_test_split(_x, _y, test_size=0.1, random_state=1, stratify=_y) 
    return x, y

# 一括処理のためにデータセットの辞書を作成
dataset = {'iris': load_iris_data(), 'mnist': load_mnist_data(), 'fashon-mnist': load_fashion_mnist_data()}

In [9]:
# 一括処理のためにモデルの辞書を作成
model = {
    # k近傍法のモデル
    'kNN(k=3)':
    KNeighborsClassifier(n_neighbors=3, # k を指定 (デフォルトは 5)
                         weights='uniform',  # 距離を考慮しない(uniform:デフォルト)、する(distance)
                         algorithm='auto', # 近傍点計算アルゴリズム (auto:デフォルト,ball_tree,kd_tree,brute)
                         leaf_size=30,  # ball_tree,kd_tree指定時のリーフサイズの設定 (デフォルトは 30)
                         p=2),  # 距離計算の次元 (2:デフォルト、1)
    # svm (kernel="linear", C=1.0) のモデル
    'SVC(kernel="linear", C=1)':
    svm.SVC(kernel="linear", C=1, max_iter=100000, verbose=True, random_state=1),
    # svm (kernel="rbf", C=1) のモデル
    'SVC(kernel="rbf", C=1)':
    svm.SVC(kernel="rbf", C=1, max_iter=100000, verbose=True, random_state=1),
}

In [13]:
# 辞書に格納したデータセットそれぞれについて性能を確認
for dataset_key in dataset.keys():
    # データを学習用と検証用に分割
    x, y = dataset[dataset_key]
    x_train, x_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.25, random_state=1, stratify=y) # 検証用データに25%を割当て

    print(f'## dataset:{dataset_key} ',
          f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')


## dataset:iris  x_train:75 x_test:25 y_train:75 y_test:25
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.92 test_data: 1.0
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.98667 test_data: 1.0
[LibSVM]dataset:iris model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.93333 test_data: 0.96
## dataset:mnist  x_train:5250 x_test:1750 y_train:5250 y_test:1750
dataset:mnist model:kNN(k=3) accuracy_score: train_data: 0.96686 test_data: 0.93371
[LibSVM]dataset:mnist model:SVC(kernel="linear", C=1) accuracy_score: train_data: 1.0 test_data: 0.90971
[LibSVM]dataset:mnist model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.9861 test_data: 0.94971
## dataset:fashon-mnist  x_train:5250 x_test:1750 y_train:5250 y_test:1750
dataset:fashon-mnist model:kNN(k=3) accuracy_score: train_data: 0.89295 test_data: 0.80286
[LibSVM]dataset:fashon-mnist model:SVC(kernel="linear", C=1) accuracy_score: train_data: 1.0 test_data: 0.79429
[LibSVM]dataset:fas

In [14]:
# 辞書に格納したデータセットそれぞれについて性能を確認
for dataset_key in dataset.keys():
    # データを学習用と検証用に分割
    x, y = dataset[dataset_key]
    x_train, x_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.25, random_state=1, stratify=y) # 検証用データに25%を割当て
    print(f'## dataset:{dataset_key} ',
          f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # データ標準化なしで性能を測定
    print('# no scaling')
    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')

    # データを標準化
    print('# with scaling')
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')

## dataset:iris  x_train:75 x_test:25 y_train:75 y_test:25
# no scaling
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.92 test_data: 1.0
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.98667 test_data: 1.0
[LibSVM]dataset:iris model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.93333 test_data: 0.96
# with scaling
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.93333 test_data: 0.96
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.96 test_data: 1.0
[LibSVM]dataset:iris model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.94667 test_data: 1.0
## dataset:mnist  x_train:5250 x_test:1750 y_train:5250 y_test:1750
# no scaling
dataset:mnist model:kNN(k=3) accuracy_score: train_data: 0.96686 test_data: 0.93371
[LibSVM]dataset:mnist model:SVC(kernel="linear", C=1) accuracy_score: train_data: 1.0 test_data: 0.90971
[LibSVM]dataset:mnist model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 